# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""12218784699474…","""../../../../da…","""16572993644997…","""17615327437218…","""13748744088833…",false
1.4856e19,"""11664559491605…","""../../../../da…","""15323330438582…","""17615327437218…","""11405312846311…",true
5.6458e17,"""17818714246510…","""../../../../da…","""40562624011471…","""34950384669302…","""13748744088833…",false
4.0723e18,"""14034663189207…","""../../../../da…","""15323330438582…","""70736739346241…","""13748744088833…",false
1.4677e19,"""10444893483847…","""../../../../da…","""16572993644997…","""70736739346241…","""17466373686993…",true
1.6330e19,"""16087207103916…","""../../../../da…","""40562624011471…","""83932373740557…",null,false
8.7675e18,"""13304530244605…","""../../../../da…","""40562624011471…","""70736739346241…","""17466373686993…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""86606204180755…","""../../../../da…","""54581766898083…","""FirstMortgage3…","""68189799491514…","""Rejected"""
1.4856e19,"""17757176471861…","""../../../../da…","""84491316786583…","""FirstMortgage3…","""17477144953442…","""Accepted"""
5.6458e17,"""52840798808264…","""../../../../da…","""97268020857330…","""MoneyMarketSav…","""68189799491514…","""Rejected"""
4.0723e18,"""17484896181961…","""../../../../da…","""84491316786583…","""BasicChecking""","""68189799491514…","""Rejected"""
1.4677e19,"""95983250069319…","""../../../../da…","""54581766898083…","""BasicChecking""","""63907070653938…","""Accepted"""
1.6330e19,"""13577353539760…","""../../../../da…","""97268020857330…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""65719760475481…","""../../../../da…","""97268020857330…","""BasicChecking""","""63907070653938…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""10604228058608…","""../../../../da…","""55081110609466…","""FirstMortgage3…","""59958942236058…","""Rejected"""
1.4856e19,"""17158792168735…","""../../../../da…","""14908757140887…","""FirstMortgage3…","""78621541948389…","""Accepted"""
5.6458e17,"""31343449231457…","""../../../../da…","""11820534906403…","""MoneyMarketSav…","""59958942236058…","""Rejected"""
4.0723e18,"""96928619995131…","""../../../../da…","""14908757140887…","""BasicChecking""","""59958942236058…","""Rejected"""
1.4677e19,"""60687904926235…","""../../../../da…","""55081110609466…","""BasicChecking""","""35179578350898…","""Accepted"""
1.6330e19,"""63544432848266…","""../../../../da…","""11820534906403…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""37642989087553…","""../../../../da…","""11820534906403…","""BasicChecking""","""35179578350898…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""40196945896493…","""13798084114447…","""FirstMortgage3…","""30588241176168…","""Rejected"""
1.4856e19,"""45134561734569…","""64503284393415…","""FirstMortgage3…","""61929722680852…","""Accepted"""
5.6458e17,"""15945104598315…","""99537567147413…","""MoneyMarketSav…","""30588241176168…","""Rejected"""
4.0723e18,"""17149883280324…","""64503284393415…","""BasicChecking""","""30588241176168…","""Rejected"""
1.4677e19,"""17919991720699…","""13798084114447…","""BasicChecking""","""15922805819319…","""Accepted"""
1.6330e19,"""17609449331077…","""99537567147413…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""35015719473000…","""99537567147413…","""BasicChecking""","""15922805819319…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'